In [19]:
# --- Standard Library Imports ---
from pathlib import Path  # Object-oriented filesystem paths
import warnings           # Handling warning messages

# --- Data Manipulation & Linear Algebra ---
import pandas as pd
import numpy as np

# --- Scikit-Learn: Metrics & Preprocessing ---
from sklearn.metrics import mean_squared_error       # Evaluation metric
from sklearn.preprocessing import LabelEncoder       # Categorical encoding
from sklearn.model_selection import StratifiedGroupKFold # specific CV strategy for grouped data

# --- Modeling ---
from catboost import Pool, CatBoostRegressor         # Gradient Boosting algorithm

# Configuration: Ignore benign warnings to keep the notebook clean
warnings.filterwarnings('ignore', category=RuntimeWarning)
warnings.filterwarnings('ignore', category=UserWarning)

In [20]:
pd.set_option('display.max_columns',None)

In [ ]:
# --- Raw Data Directories ---
# Directories containing granular flight sensor data
TRAIN_FLIGHTS_DIR = '../prc-2025-datasets/flights_train'
RANK_FLIGHTS_DIR  = '../prc-2025-datasets/flights_rank'
FINAL_FLIGHTS_DIR = '../prc-2025-datasets/flights_final'

# --- Flight Metadata Lists ---
# Parquet files containing the list of flights for each phase
FLIGHTLIST_TRAIN = '../prc-2025-datasets/flightlist_train.parquet'
FLIGHTLIST_RANK  = '../prc-2025-datasets/flightlist_rank.parquet'
FLIGHTLIST_FINAL = '../prc-2025-datasets/flightlist_final.parquet'

# --- Target & Submission Files ---
# Fuel consumption data (Ground truth for train, Submission templates for rank/final)
FUEL_TRAIN = '../prc-2025-datasets/fuel_train.parquet'
FUEL_RANK  = '../prc-2025-datasets/fuel_rank_submission.parquet'
FUEL_FINAL = '../prc-2025-datasets/fuel_final_submission.parquet'

# --- Auxiliary Data ---
# Airport information (likely coordinates, runway info, etc.)
AIRPORTS = '../prc-2025-datasets/apt.parquet'

# --- Output Paths ---
# Path to load the preprocessed training dataset
PREPARED_TRAIN_DATA = './prep_train_acropole.csv'
PREPARED_RANK_DATA = './prep_rank_acropole.csv'
PREPARED_FINAL_DATA = './prep_final_acropole.csv'

In [22]:
TRAIN_FLIGHTS_DIR = Path(TRAIN_FLIGHTS_DIR)
RANK_FLIGHTS_DIR = Path(RANK_FLIGHTS_DIR)
FINAL_FLIGHTS_DIR = Path(FINAL_FLIGHTS_DIR)

In [23]:
# load all necessary data
train_flightlist = pd.read_parquet(FLIGHTLIST_TRAIN)
rank_flightlist = pd.read_parquet(FLIGHTLIST_RANK)
final_flightlist = pd.read_parquet(FLIGHTLIST_FINAL)
airports = pd.read_parquet(AIRPORTS)
train_fuel = pd.read_parquet(FUEL_TRAIN)
rank_fuel = pd.read_parquet(FUEL_RANK)
final_fuel = pd.read_parquet(FUEL_FINAL)

In [ ]:
# read the prepared train data
prep_train = pd.read_csv(PREPARED_TRAIN_DATA)

In [ ]:
def haversine(lat_lon1, lat_lon2):
    """
    Calculates the great-circle distance between two points on the earth's surface
    using the Haversine formula.

    Parameters
    ----------
    lat_lon1 : tuple or list of float
        The first coordinate pair in degrees: (latitude, longitude).
    lat_lon2 : tuple or list of float
        The second coordinate pair in degrees: (latitude, longitude).

    Returns
    -------
    float
        The distance between the two points in meters.
    """
    # Radius of the Earth in kilometers
    R = 6371.0
    
    # Unpack and convert latitude and longitude from degrees to radians
    lat1, lon1 = map(np.radians, lat_lon1)
    lat2, lon2 = map(np.radians, lat_lon2)
    
    # Apply Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    # Return distance converted to meters (km * 1000)
    return R * c * 1000

In [ ]:
def extra_features(df, datetime_cols=["timestamp"]):
    """
    Generates additional geospatial and temporal features for the flight data.
    
    Processing Steps:
    1. Calculates Great Circle Distance between Origin and Destination.
    2. Converts specified timestamp columns to datetime objects.
    3. Calculates segment duration in minutes.
    4. Drops original raw timestamp columns.

    Parameters
    ----------
    df : pd.DataFrame
        Input dataframe containing flight logs. Must include:
        ['orig_lat', 'orig_long', 'dest_lat', 'dest_long', 'seg_start', 'seg_end']
    datetime_cols : list, optional
        List of columns to convert to datetime objects and subsequently drop.
        Default is ["timestamp"].

    Returns
    -------
    pd.DataFrame
        The modified dataframe with new features and dropped timestamp columns.
    """
    
    # 1. Calculate Full Flight Distance (Origin -> Destination) in meters
    # Uses the previously defined 'haversine' function row-by-row
    df['full_flight_distance'] = df.apply(
        lambda row: haversine(
            (row['orig_lat'], row['orig_long']),
            (row['dest_lat'], row['dest_long'])
        ),
        axis=1
    )
    
    # 2. Standardize Datetime Columns
    # Convert string timestamps to datetime objects (UTC)
    for dt_col in datetime_cols:
        if dt_col in df.columns:
            df[dt_col] = pd.to_datetime(df[dt_col], format='ISO8601', utc=True)
   
    # 3. Calculate Segment Duration
    # Difference between segment end and start, converted to minutes
    # Note: Ensure 'seg_end' and 'seg_start' are already in datetime format 
    # (or included in the datetime_cols list if they are strings initially)
    df['seg_duration'] = (df['seg_end'] - df['seg_start']).dt.total_seconds() / 60

    # 4. Cleanup
    # Remove the original datetime columns (models usually require numerical/encoded inputs)
    result_df = df.drop(columns=datetime_cols)
    
    return result_df

In [ ]:
# Define the temporal columns representing key flight events
# These are passed to the function to be converted/used for duration calc, then dropped
dt_feat = ['takeoff', 'landed', 'seg_start', 'seg_end']

# Apply feature engineering:
# - Calculates Great Circle Distance (Origin -> Destination)
# - Computes segment duration (in minutes)
# - Removes the raw datetime columns listed in 'dt_feat'
train_interact = extra_features(prep_train, dt_feat)

In [ ]:
# List of categorical features to be encoded
# Includes operational data (Origin/Dest), technical specs (Engine, Wake), and classification codes
cols_to_encode = [
    'aircraft_type', 'origin_icao', 'destination_icao',
    'WAKE', 'ENG_NAME', 'ENG_MAN', 'ENG_MODEL', 'Manufacturer',
    'Physical_Class_Engine', 'AAC', 'AAC_minimum', 'AAC_maximum', 'ADG',
    'TDG', 'Main_Gear_Config', 'ICAO_WTC', 'Class', 'FAA_Weight', 'CWT',
    'One_Half_Wake_Category', 'Two_Wake_Category_Appx_A',
    'Two_Wake_Category_Appx_B', 'SRS', 'flaps_type', 'engine_type',
    'engine_mount', 'engine_default', 'eng_type', 'eng_manufacturer',
    'fuel_engine', 'fuel_aircraft'
]

# Dictionary to store the fitted encoder objects for each column
# This allows consistent transformation of Test/Rank data later
encoder_dict = {}

for col in cols_to_encode:
    # Initialize a new LabelEncoder for this column
    sp_enc = LabelEncoder()
    
    # Fit the encoder on the training data
    # Note: We cast to 'str' to handle potential NaN values or mixed types gracefully
    sp_enc.fit(train_interact[col].astype('str'))
    
    # Store the encoder
    encoder_dict[col] = sp_enc

In [ ]:
# Iterate through the list of categorical columns
for col in cols_to_encode:
    # Retrieve the specific encoder fitted for this column
    encoder = encoder_dict[col]
    
    # Transform the text data to integers
    # .astype('str') ensures we handle NaNs as the string 'nan', consistent with the .fit() step
    train_interact[col] = encoder.transform(train_interact[col].astype('str'))

In [ ]:
# --- 1. Feature Selection ---

# List of columns to remove from the feature set
# These include unique identifiers, redundant names, and raw date strings
cols_to_drop = [
    'flight_id', 'origin_name', 'destination_name', 'flight_date', 'MODEL',
    'aircraft', 'eng_name', 'eng_uid', 'flight_start', 'flight_end',
    'takeoff_date', 'land_date'
]

# Create Feature Matrix (X)
# Drop the target ('fuel_kg') and the unused columns
X = train_interact.drop(columns=['fuel_kg'] + cols_to_drop)

# Create Target Vector (y)
y = train_interact['fuel_kg']

# Keep track of categorical indices for the CatBoost model
cat_cols = cols_to_encode

# --- 2. Stratification Key ---

# Create a custom column to guide the Cross-Validation split.
# We combine 'missing_segment' status and 'aircraft_type' to ensure
# the validation set reflects the diversity of data quality and aircraft models.
stratifier = train_interact['missing_segment'].astype('str') + '_' + train_interact['aircraft_type'].astype('str')

In [ ]:
# List of features selected through iterative experimentation and feature importance analysis.
# These variables yielded the best RMSE performance during cross-validation.

sel_ind = [
    # --- Segment Dynamics & Aggregates ---
    'rawseg_diff_mean', 'flight_DE_ct', 'seg_ENR_dur', 'all_seg_phase_dur',
    'start_longitude', 'min_longitude', 'dest_long',
    'end_altitude', 'mean_altitude', 'sum_altitude', 'total_climb_height',
    'sum_groundspeed', 'sum_CAS', # CAS: Calibrated Airspeed
    
    # --- Fuel & Engine Telemetry Proxies ---
    'unscaled_approx_seg_fuel', 'nancount_mach', 
    'sum_fuel_flow', 'sum_fuel', 'sum_drag', 
    'start_thrust', 'sum_thrust', 'end_auth_score',
    
    # --- Previous Segment/Accumulated Fuel Metrics ---
    'sum_cl_fuel', 'sum_enr_fuel', 'sum_dist_from_ades', 
    'start_acp_fuel', 'max_acp_fuel', 'sum_acp_fuel', 'sum_acp_fuelflow',
    
    # --- Static Aircraft Specifications ---
    'Tail_Height_at_OEW_ft',
    'MALW_lb',   # Maximum Aircraft Landing Weight
    'mtow',      # Maximum Takeoff Weight
    'wing_mac',  # Mean Aerodynamic Chord
    'drag_gears',
    
    # --- Engine Characteristics & Emissions ---
    'eng_bpr',        # Engine Bypass Ratio
    'eng_ei_co_co',   # Emission Index (Carbon Monoxide)
    'eng_ei_co_app', 
    'eng_ei_nox_co',
    
    # --- Calculated Interaction Features ---
    'full_flight_distance', 
    'seg_duration'
]

In [ ]:
# Optimal hyperparameters derived from Optuna optimization
# Target Benchmark: 5-Fold CV RMSE ~= 213.90
hyp = {
    # --- Ensemble Structure ---
    'iterations': 972,                # Total number of trees
    'max_depth': 6,                   # Depth of the tree (controls interaction complexity)
    'learning_rate': 0.05060963202084587, # Step size shrinkage used in update to prevent overfitting
    
    # --- Regularization & Sampling ---
    'l2_leaf_reg': 1.822777238628044, # L2 regularization term on cost function
    'min_data_in_leaf': 78,           # Minimum number of samples in a leaf (smooths the model)
    
    # --- Stochastic Features (Randomness) ---
    'colsample_bylevel': 0.6035766248956298, # Fraction of features selected at each tree level
    'subsample': 0.46633222868063806,        # Fraction of data samples used for each tree
    'bootstrap_type': 'MVS'                  # 'Minimum Variance Sampling' (CatBoost specific sampling)
}

In [ ]:
# --- 1. CV Setup ---
# Initialize the cross-validator
scv = StratifiedGroupKFold(n_splits=5)

# Containers to store results and data for post-analysis
fit_models = []
ll_scores = []      # Hold-out (Validation) scores
train_scores = []   # Training scores (to check for overfitting)

# Storage for debugging/analysis (dictionaries mapped by fold index)
train_features, train_targets = {}, {}
valid_features, valid_targets = {}, {}
valid_preds = {}

ind = 0

# --- 2. Training Loop ---
# Split data ensuring Flight IDs do not leak between Train and Test
for train_index, test_index in scv.split(X, stratifier, groups=train_interact['flight_id']):
    
    # A. Split Data
    X_train, X_test = X.iloc[train_index].copy(), X.iloc[test_index].copy()
    y_train, y_test = y.iloc[train_index], y.iloc[test_index] 

    # Store raw splits for later analysis
    train_features[ind] = X_train
    train_targets[ind] = y_train
    valid_features[ind] = X_test
    valid_targets[ind] = y_test
    
    # B. Feature Subset Selection
    # Reduce feature space to the selected 'sel_ind' list
    preppedtrain_sel = X_train[sel_ind]
    preppedtest_sel = X_test[sel_ind]

    # Identify indices of categorical features within the selected subset
    # CatBoost requires specific indices for categorical handling
    cat_features = [i for i, col in enumerate(sel_ind) if col in cat_cols]
            
    # Create optimized CatBoost Pools
    train_pool = Pool(preppedtrain_sel, y_train, cat_features=cat_features)
    test_pool = Pool(preppedtest_sel, y_test, cat_features=cat_features)
    
    # C. Model Initialization & Fitting
    model = CatBoostRegressor(
        objective='RMSE',
        task_type='CPU',
        random_state=42,
        silent=True,
        **hyp  # Unpack optimal hyperparameters
    )
    
    model.fit(train_pool, eval_set=test_pool)
    fit_models.append(model)

    # D. Prediction & Post-Processing
    # Predict on Validation Set
    y_pred = model.predict(preppedtest_sel)
    
    # Physics Constraint: Fuel cannot be negative. 
    # Clip predictions to the minimum observed fuel in the training set (or global minimum)
    # Note: Ensure 'train_fuel' is defined globally, otherwise replace with y_train.min()
    min_fuel_threshold = train_fuel['fuel_kg'].min() 
    y_pred = np.where(y_pred < 0, min_fuel_threshold, y_pred)

    valid_preds[ind] = y_pred
    ind += 1

    # Predict on Training Set (for bias/variance analysis)
    train_pred = model.predict(preppedtrain_sel)
    train_pred = np.where(train_pred < 0, min_fuel_threshold, train_pred)
    
    # E. Evaluation
    ind_cv = mean_squared_error(y_test, y_pred, squared=False) # RMSE
    tr_cv = mean_squared_error(y_train, train_pred, squared=False)

    ll_scores.append(ind_cv)
    train_scores.append(tr_cv)

# --- 3. Summary Output ---
print(f'Mean 5-Fold RMSE Train: {np.mean(train_scores):.4f}') 
print(f'Mean 5-Fold RMSE Test:  {np.mean(ll_scores):.4f}')

mean 5fold rmse train:  135.50833061642936
mean 5fold rmse test:  213.89598165883922


In [ ]:
# Load the pre-processed ranking/test data
# (Ensure PREPARED_RANK_DATA is defined in your config section, similar to PREPARED_TRAIN_DATA)
prep_rank = pd.read_csv(PREPARED_RANK_DATA)

# 1. Apply Feature Engineering
# Generate the same interaction features (Distances, Durations) as the training set
rank_interact = extra_features(prep_rank, dt_feat)

# 2. Apply Categorical Encoding
# We iterate through the encoders fitted on the training set
for col in cols_to_encode:
    # Optimization: Only transform columns that are actually part of the final feature set ('sel_ind')
    if col in sel_ind:
        # Transform using the stored dictionary to ensure mapping consistency (Train vs Rank)
        # .astype('str') handles potential data type mismatches or NaNs
        rank_interact[col] = encoder_dict[col].transform(rank_interact[col].astype('str'))

In [ ]:
# --- Ensemble Prediction ---

# Generate predictions from each of the 5 cross-validated models
preds = [
    np.clip(
        model.predict(rank_interact[sel_ind]),  # Predict using selected features
        a_min=train_fuel['fuel_kg'].min(),      # Lower bound: Min observed training fuel
        a_max=None                              # Upper bound: No restriction
    ) 
    for model in fit_models
]

# --- Averaging ---

# Calculate the mean prediction across all folds (Model Ensembling)
# This reduces the variance of the error and generally improves generalization
pred = sum(preds) / len(preds)

# Assign the final averaged prediction to the ranking dataframe
rank_fuel['fuel_kg'] = pred

In [74]:
rank_fuel['fuel_kg'].describe()

count    24289.000000
mean       416.292326
std        745.771351
min          0.453592
25%         92.157918
50%        151.403112
75%        394.877050
max      12200.323908
Name: fuel_kg, dtype: float64

In [ ]:
# Save the predictions for submission
rank_fuel.to_parquet('wise-watermelon_v26.parquet')


## FINAL

In [107]:
prep_final = pd.read_csv('/kaggle/input/prc2025-accessories/prep_final_acropole/prep_final_acropole.csv')


/tmp/ipykernel_47/3462117581.py:1: DtypeWarning: Columns (347,348,349,350,351,352,353,354,361,362,363,364,365,366,367,373,374,376,377,378,379,380,391,392,394,395,396,397,398,399,401,404,423,432,433,435,447,448,449,450,477,479) have mixed types. Specify dtype option on import or set low_memory=False.
  prep_final = pd.read_csv('/kaggle/input/prc2025-accessories/prep_final_acropole/prep_final_acropole.csv')


In [ ]:
final_interact = extra_features(prep_final,dt_feat)

In [ ]:
for col in cols_to_encode:
    if col in sel_ind:
        print(col)
        final_interact[col] = encoder_dict[col].transform(final_interact[col].astype('str'))


In [110]:
preds_final = [np.clip(model.predict(final_interact[sel_ind]),
                a_min=train_fuel['fuel_kg'].min(),a_max=None) for model in fit_models]
preds_final = sum(preds_final) / len(preds_final)

In [111]:
final_fuel['fuel_kg'] = preds_final

In [112]:
final_fuel['fuel_kg'].describe()

count    61745.000000
mean       512.410579
std        830.672808
min          0.453592
25%        107.267244
50%        360.601572
75%        471.461127
max      13004.959405
Name: fuel_kg, dtype: float64

In [113]:
final_fuel.to_parquet('wise-watermelon_final.parquet')